In [3]:
# =============================import=============================
import heapq

import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import numpy as np
import time
import os
import hyperloglog
from pympler import asizeof

def index_of_element(item_list,item):
    for i,element in enumerate(item_list):
        if element[1]==item:
            return i
    return None

# =============================change parameter=============================
dataset='kosarak'
heavy_ratio=1/5000

# =============================set dataset path and file=============================
filename=dataset+'.dat'
filepath=r"..\..\dataset\kosarak"
gr_file_name='kosarak_ground_truth.csv'
src_data=os.path.join(filepath,filename)
    # dataset
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth
    
# =============================Initialize=============================
w=64
d=79
size=[512,1024,2048,3172,4096,5120,6144,7168,8192,9216,10240]
topk=[512,1024,2048,3172,4096,5120,6144,7168,8192,9216,10240]

    # cs_heap[i]=[count,id]

# =============================Stream processing=============================

for i in size[:2]:
    TC.Set_default(w,d,i,i)
        # set width, depth, size of Sk, random seed of hash
        # Config.width, Config.depth
    Sk_head=[DS.Head(0) for j in range(TC.depth)]
    Sketch=np.zeros((TC.depth,TC.width),dtype='int32')
    hll = hyperloglog.HyperLogLog(0.05)
    #Top_dict=dict()    
    e_min=DS.Tail("",0)
    cs_heap=[]
    total_count=0
    
    start=time.time()
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print('EOF')
                break
            else:
                #item_count-=1
                total_count+=1
                item=DS.Tail(e,1)
                find=index_of_element(cs_heap,item.ID)
                if find!=None:
                    # e in Top
                    cs_heap[find][0]+=1
                    heapq.heapify(cs_heap)
                else:
                    if len(cs_heap)<TC.size:
                        heapq.heappush(cs_heap,[1,item.ID])
                    else:
                        # cs_heap is full
                        hll.add(item.ID)
                        TF.UpdateSk(item,Sk_head,Sketch)
            if TC.e_max.count>e_min.count:                
                min_count,min_id=heapq.heappop(cs_heap)
                e_min=DS.Tail(min_id,min_count)
                TF.BringBack(e_min,Top_dict,Sk_head,Sketch)
    end=time.time()
    

NameError: name 'Top_dict' is not defined